In [ ]:
# Standard Kaggle first cell

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

<h1> Baseline Article Classifier </h1>

<h2> Set up required data, packages, functions  </h2>

In [ ]:
# Read in dataframe (validation data)
df=pd.read_csv("../input/test-val/val_data.tsv", delimiter= "\t")
print(df['uid'])
print(df.dtypes)

In [ ]:
# Install necessary packages
!pip install git+https://github.com/JohnGiorgi/seq2rel-ds.git

In [ ]:
# Import necessary functions
from seq2rel_ds.common.util import query_pubtator

<h2>In these next two cells, we effectively generate predicted class for each ID</h2>

In [ ]:
# Get pubtator annotations for pmids in validation set
pmids= list(df['uid'])
annotations = query_pubtator(pmids=pmids, concepts=["gene"], skip_malformed=True)

In [ ]:
# Use the annotations to label articles based on whether more than one unique gene is mentioned
baseline_labels = {}
for pmid, ann in annotations.items():
    label = 1 if len(ann.clusters) > 1 else 0
    baseline_labels[pmid] = label
print(baseline_labels)

<h2> Here we add predictions to dataframe, then evaluate metrics </h2>

In [ ]:
# Add new column with labels for each ID (from dictionary generated in last cell)
# Necessary to convert uid to str as this is their type in dictionary
df['predictions']=(df['uid'].astype(str)).map(baseline_labels)

In [ ]:
# Inspect new dataframe
print(df)

In [ ]:
# Calcualte metrics
from sklearn.metrics import classification_report
from sklearn.metrics import matthews_corrcoef

print(classification_report(df['class'], df['predictions']))
print(matthews_corrcoef(df['class'], df['predictions']))